In [44]:
import pandas as pd

In [45]:
match_path = "resources/data.csv"
fighter_path = "resources/raw_fighter_details.csv"

In [46]:
match_data = pd.read_csv("resources/data.csv")

In [47]:
match_columns = ['R_fighter','B_fighter','date','location','Winner','weight_class']

match_df = match_data[match_columns]
match_df.columns = ["red","blue","date","location","win_color","weight_class"]
match_df["winner"] = ""
match_df["loser"] = ""

C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
for index, row in match_df.iterrows():
    if row[4] == "Red":
        row[6] = row[0]
        row[7] = row[1]
    elif row[4] == "Blue":
        row[7] = row[0]
        row[6] = row[1]

# Remove draw data
match_df = match_df[match_df["winner"] !=""]

In [49]:
transformed_match_df = match_df[["date","location","weight_class","winner","loser"]]
transformed_match_df


# Dan load this df to the fight_table

,date,location,weight_class,winner,loser
0,2019-06-08,"Chicago, Illinois, USA",Bantamweight,Henry Cejudo,Marlon Moraes
1,2019-06-08,"Chicago, Illinois, USA",Women's Flyweight,Valentina Shevchenko,Jessica Eye
2,2019-06-08,"Chicago, Illinois, USA",Lightweight,Tony Ferguson,Donald Cerrone
3,2019-06-08,"Chicago, Illinois, USA",Bantamweight,Petr Yan,Jimmie Rivera
4,2019-06-08,"Chicago, Illinois, USA",Heavyweight,Blagoy Ivanov,Tai Tuivasa
...,...,...,...,...,...
5139,1993-11-12,"Denver, Colorado, USA",Open Weight,Gerard Gordeau,Kevin Rosier
5140,1993-11-12,"Denver, Colorado, USA",Open Weight,Ken Shamrock,Patrick Smith
5141,1993-11-12,"Denver, Colorado, USA",Open Weight,Royce Gracie,Art Jimmerson
5142,1993-11-12,"Denver, Colorado, USA",Open Weight,Kevin Rosier,Zane Frazier


In [50]:
fighter_data = pd.read_csv(fighter_path)

In [51]:
fighter_df = fighter_data.iloc[:,:3]

# Conversion
1 ft = 30.48 cm/ 1 in = 2.54 cm

1 lbs = 0.454 kg

In [52]:
for index, row in fighter_df.iterrows():
    # to ignore the blank (not sure why blank has three length)
    if len(str(row[1])) >3:
        row[1] = int(str(row[1]).split()[0].rstrip("'")) *30.48 + int(str(row[1]).split()[1].rstrip('"'))*2.54
    else :
        row[1] = "Not available"
        
    if len(str(row[2])) > 3:
        row[2] = int(row[2].split()[0]) * 0.454
    else :
        row[2] = "Not available"
        
fighter_df.columns = ["fighter_name", "height", "weight"]

In [53]:
# pick up the data where height and weight are available

fighter_df = fighter_df[(fighter_df["height"] != "Not available") & (fighter_df["weight"] != "Not available") ]

In [54]:
transformed_fighter_df = fighter_df.reset_index()[["fighter_name", "height", "weight"]]

In [55]:
transformed_fighter_df

# Dan load this df to the fighter_table

,fighter_name,height,weight
0,AJ Fonseca,162.56,65.83
1,AJ Matthews,180.34,83.99
2,AJ McKee,177.8,65.83
3,AJ Siscoe,170.18,61.29
4,Aalon Cruz,182.88,65.83
...,...,...,...
3039,Zelim Imadaev,182.88,77.18
3040,Zhang Lipeng,180.34,70.37
3041,Zoila Frausto,162.56,61.29
3042,Zu Anyanwu,185.42,113.954


## get the information using web-scraping

In [13]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from datetime import datetime

In [14]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [15]:
url = 'http://www.ufcstats.com/statistics/events/completed?page=all'
browser.visit(url)
time.sleep(10)

In [16]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [17]:
match_info_tag = soup.find_all('i', class_='b-statistics__table-content')

match_info = []
for each in match_info_tag:
    match_info.append({"url":each.a['href'],"date":datetime.strptime(each.span.text.strip(),"%B %d, %Y").strftime("%Y-%m-%d")})

len(match_info)

530

In [18]:
location_tag = soup.find_all('td', class_='b-statistics__table-col_style_big-top-padding')

location_info =[]
for each in location_tag:
    location_info.append(each.text.strip())

len(location_info)

530

In [19]:
# Combine two lists into match_info

for i in range(0,len(match_info)):
    match_info[i]["location"] = location_info[i]


In [20]:
new_fights = {"date":[],"location":[],"weight_class":[],"winner":[],"loser":[]}

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for each in match_info:
    if each["date"] < "2020-08-25" and each["date"] > "2019-06-08":
        url = each["url"]
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        info_tags = soup.find_all('tr')
        
        for i in range(1,len(info_tags)):
            if info_tags[i].find_all("td")[0].text.strip() == 'win':
            # remove draw or other case
                new_fights["date"].append(each["date"])
                new_fights["location"].append(each["location"])
                new_fights["winner"].append(info_tags[i].find_all("td")[1].text.strip().split("\n")[0].strip())
                new_fights["loser"].append(info_tags[i].find_all("td")[1].text.strip().split("\n")[5].strip())
                new_fights["weight_class"].append(info_tags[i].find_all("td")[6].text.strip())

 

In [56]:
new_df = pd.DataFrame(new_fights)
new_df   

,date,location,weight_class,winner,loser
0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,Frankie Edgar,Pedro Munhoz
1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,Mike Rodriguez,Marcin Prachnio
2,2020-08-22,"Las Vegas, Nevada, USA",Lightweight,Joe Solecki,Austin Hubbard
3,2020-08-22,"Las Vegas, Nevada, USA",Women's Flyweight,Shana Dobson,Mariya Agapova
4,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,Daniel Rodriguez,Dwight Grant
...,...,...,...,...,...
549,2019-06-22,"Greenville, South Carolina, USA",Women's Strawweight,Ashley Yoder,Syuri Kondo
550,2019-06-22,"Greenville, South Carolina, USA",Lightweight,Luis Pena,Matt Wiman
551,2019-06-22,"Greenville, South Carolina, USA",Heavyweight,Jairzinho Rozenstruik,Allen Crowder
552,2019-06-22,"Greenville, South Carolina, USA",Women's Flyweight,Molly McCann,Ariane Lipski


In [58]:
# concut match_df and new_df

all_match_df = pd.concat([new_df,transformed_match_df])
all_match_df

,date,location,weight_class,winner,loser
0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,Frankie Edgar,Pedro Munhoz
1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,Mike Rodriguez,Marcin Prachnio
2,2020-08-22,"Las Vegas, Nevada, USA",Lightweight,Joe Solecki,Austin Hubbard
3,2020-08-22,"Las Vegas, Nevada, USA",Women's Flyweight,Shana Dobson,Mariya Agapova
4,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,Daniel Rodriguez,Dwight Grant
...,...,...,...,...,...
5139,1993-11-12,"Denver, Colorado, USA",Open Weight,Gerard Gordeau,Kevin Rosier
5140,1993-11-12,"Denver, Colorado, USA",Open Weight,Ken Shamrock,Patrick Smith
5141,1993-11-12,"Denver, Colorado, USA",Open Weight,Royce Gracie,Art Jimmerson
5142,1993-11-12,"Denver, Colorado, USA",Open Weight,Kevin Rosier,Zane Frazier


In [59]:

transformed_fighter_df = transformed_fighter_df.reset_index()

transformed_fighter_df= transformed_fighter_df.rename(columns={"index":"id"})
transformed_fighter_df

,id,fighter_name,height,weight
0,0,AJ Fonseca,162.56,65.83
1,1,AJ Matthews,180.34,83.99
2,2,AJ McKee,177.8,65.83
3,3,AJ Siscoe,170.18,61.29
4,4,Aalon Cruz,182.88,65.83
...,...,...,...,...
3039,3039,Zelim Imadaev,182.88,77.18
3040,3040,Zhang Lipeng,180.34,70.37
3041,3041,Zoila Frausto,162.56,61.29
3042,3042,Zu Anyanwu,185.42,113.954


In [60]:
all_match_df = all_match_df.reset_index()
all_match_df = all_match_df.rename(columns={"index":"id"})
all_match_df 

,id,date,location,weight_class,winner,loser
0,0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,Frankie Edgar,Pedro Munhoz
1,1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,Mike Rodriguez,Marcin Prachnio
2,2,2020-08-22,"Las Vegas, Nevada, USA",Lightweight,Joe Solecki,Austin Hubbard
3,3,2020-08-22,"Las Vegas, Nevada, USA",Women's Flyweight,Shana Dobson,Mariya Agapova
4,4,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,Daniel Rodriguez,Dwight Grant
...,...,...,...,...,...,...
5610,5139,1993-11-12,"Denver, Colorado, USA",Open Weight,Gerard Gordeau,Kevin Rosier
5611,5140,1993-11-12,"Denver, Colorado, USA",Open Weight,Ken Shamrock,Patrick Smith
5612,5141,1993-11-12,"Denver, Colorado, USA",Open Weight,Royce Gracie,Art Jimmerson
5613,5142,1993-11-12,"Denver, Colorado, USA",Open Weight,Kevin Rosier,Zane Frazier


In [95]:
combined_df = all_match_df.merge(transformed_fighter_df[["id","fighter_name"]], how="inner",left_on="winner",right_on="fighter_name").merge(transformed_fighter_df[["id","fighter_name"]], how="inner",left_on="loser",right_on="fighter_name")
all_match_table_df = combined_df[["date","location","weight_class","id_y","id"]].sort_values(by="date",ascending=False)
all_match_table_df.columns = ["date","location","weight_class","winner_fighter_id","loser_fighter_id"]

In [96]:
all_match_table_df = all_match_table_df.reset_index().reset_index()
all_match_table_df = all_match_table_df[["level_0","date","location","weight_class","winner_fighter_id","loser_fighter_id"]].rename(columns={"level_0":"id"})
all_match_table_df 

,id,date,location,weight_class,winner_fighter_id,loser_fighter_id
0,0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,996,2288
1,1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,2101,1905
2,2,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,690,862
3,3,2020-08-15,"Las Vegas, Nevada, USA",Women's Strawweight,1822,256
4,4,2020-08-15,"Las Vegas, Nevada, USA",Lightweight,2913,1373
...,...,...,...,...,...,...
5469,5469,1993-11-12,"Denver, Colorado, USA",Open Weight,2513,247
5470,5470,1993-11-12,"Denver, Colorado, USA",Catch Weight,2513,1046
5471,5471,1993-11-12,"Denver, Colorado, USA",Open Weight,1687,2260
5472,5472,1993-11-12,"Denver, Colorado, USA",Open Weight,2513,1687


# Load data into Database (Postgres)

In [100]:
from sqlalchemy import create_engine
rds_connection_string = "postgres:postgres@localhost:5432/ufc_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['fighter', 'match']

In [101]:
transformed_fighter_df.to_sql(name='fighter', con=engine, if_exists='append', index=False)

In [102]:
all_match_table_df.to_sql(name='match', con=engine, if_exists='append', index=False)


In [103]:
pd.read_sql_query('select * from match', con=engine).head()

,id,date,location,weight_class,winner_fighter_id,loser_fighter_id
0,0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,996,2288
1,1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,2101,1905
2,2,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,690,862
3,3,2020-08-15,"Las Vegas, Nevada, USA",Women's Strawweight,1822,256
4,4,2020-08-15,"Las Vegas, Nevada, USA",Lightweight,2913,1373


In [104]:
pd.read_sql_query('select * from fighter', con=engine).head()

,id,fighter_name,height,weight
0,0,AJ Fonseca,162.56,65.83
1,1,AJ Matthews,180.34,83.99
2,2,AJ McKee,177.80,65.83
3,3,AJ Siscoe,170.18,61.29
4,4,Aalon Cruz,182.88,65.83


# Load data into SQL Database (sqlite)

In [105]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, ForeignKey

In [108]:
class Match(Base):
    __tablename__ = 'match'
    id = Column(Integer, primary_key=True)
    date = Column(String(255))
    location = Column(String(255))
    weight_class = Column(String(255))
    winner_fighter_id = Column(Integer, ForeignKey("fighter.id"))
    loser_fighter_id = Column(Integer, ForeignKey("fighter.id"))
    
class Fighter(Base):
    __tablename__ = 'fighter'
    id = Column(Integer, primary_key=True)
    fighter_name = Column(String(255))
    height = Column(Float)
    weight = Column(Float)
       

In [109]:
engine = create_engine('sqlite:///ufc.db')

In [110]:
Base.metadata.create_all(engine)

In [111]:
transformed_fighter_df.to_sql(name='fighter', con=engine, if_exists='append', index=False)
all_match_table_df.to_sql(name='match', con=engine, if_exists='append', index=False)


In [112]:
pd.read_sql_query('select * from fighter', con=engine).head()

,id,fighter_name,height,weight
0,0,AJ Fonseca,162.56,65.83
1,1,AJ Matthews,180.34,83.99
2,2,AJ McKee,177.80,65.83
3,3,AJ Siscoe,170.18,61.29
4,4,Aalon Cruz,182.88,65.83


In [113]:
pd.read_sql_query('select * from match', con=engine).head()

,id,date,location,weight_class,winner_fighter_id,loser_fighter_id
0,0,2020-08-22,"Las Vegas, Nevada, USA",Bantamweight,996,2288
1,1,2020-08-22,"Las Vegas, Nevada, USA",Light Heavyweight,2101,1905
2,2,2020-08-22,"Las Vegas, Nevada, USA",Welterweight,690,862
3,3,2020-08-15,"Las Vegas, Nevada, USA",Women's Strawweight,1822,256
4,4,2020-08-15,"Las Vegas, Nevada, USA",Lightweight,2913,1373
